In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec

1.8.1
0.8.0a0+e4e171a


In [3]:
import torch
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/koen/.cache/torch/hub/master.zip


['resnest101',
 'resnest200',
 'resnest269',
 'resnest50',
 'resnest50_fast_1s1x64d',
 'resnest50_fast_1s2x40d',
 'resnest50_fast_1s4x24d',
 'resnest50_fast_2s1x64d',
 'resnest50_fast_2s2x40d',
 'resnest50_fast_4s1x64d',
 'resnest50_fast_4s2x40d']

In [23]:
cfg = Config(
#     n_mels = 128,
#     f_min = 20,
#     f_max = 16000,
#     n_fft = 2048,
#     hop_length = 512,
    
    noise_nsr_dbs = [35, 30, 20, 10],
    site='SSW', 
    use_neptune=True, 
    n_epochs=15, 
    bs=32, 
    lr=1e-2, 
    model='resnest50-mixed-spec',
    scheduler='torch.optim.lr_scheduler.OneCycleLR'
)

In [24]:
cfg.as_dict()

{'site': 'SSW',
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_workers': 12,
 'training_dataset_size': 10752,
 'duration': 5,
 'use_noise': True,
 'noise_nsr_dbs': [35, 30, 20, 10],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'use_neptune': True,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'lr': 0.01,
 'n_epochs': 15,
 'model': 'resnest50-mixed-spec',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'torch.nn.BCEWithLogitsLoss',
 'scheduler': 'torch.optim.lr_scheduler.OneCycleLR',
 'labels': ['amecro',
  'amegfi',
  'amerob',
  'balori',
  'belkin1',
  'bkcchi',
  'blujay',
  'cangoo',
  'chswar',
  'comgra',
  'comyel',
  'dowwoo',
  'eastow',
  'eawpew',
  'gockin',
  'grycat',
  '

In [25]:
prep = MixedSig2Spec(cfg)
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# 

main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)

# posp = torch.nn.Sequential(
#     torch.nn.Linear(in_features=2048, out_features=1024, bias=True),
#     torch.nn.ReLU(),
#     torch.nn.Linear(in_features=1024, out_features=len(cfg.labels), bias=True)
# )

posp = torch.nn.Linear(in_features=2048, 
                       out_features=len(cfg.labels), bias=True)

# for param in posp.parameters():
#     param.requires_grad = True

main_model.fc = posp

model = torch.nn.Sequential(prep, main_model)

# for param in model.parameters():
#     param.requires_grad = False
# for m in [model[1].layer3, model[1].layer4, model[1].avgpool, model[1].fc]:
#     for param in m.parameters():
#         param.requires_grad = True


model = model.cuda()




# model

# model = torch.nn.Sequential(Sig2Spec(cfg, forward_as_image=True), preprocess, rn_model)
# model[1].fc = torch.nn.Linear(in_features=2048, out_features=len(cfg.labels), bias=True)

Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [26]:
# model[1].layer3, model[1].layer4, model[1].avgpool, model[1].fc

In [27]:
lrn = Learner("resnest50-mixed-spec", cfg, model)

In [ ]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-57


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

2021-05-03 13:54:34 epoch:  12 train loss: 0.131673 train f1: 0.457355 valid loss: 0.084208 valid f1: 0.365306


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-03 13:56:54 epoch:  13 train loss: 0.129720 train f1: 0.461941 valid loss: 0.095428 valid f1: 0.321839


  0%|          | 0/336 [00:00<?, ?it/s]

In [49]:
# lrn.name = 'resnest50-mixed-spec-half-frozen-20210502-144336-latest'

In [50]:
# lrn.save_checkpoint(50, 0.097231, 0.234157)

In [10]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,349,30718,2112,421,0.216032
2,0.2,258,32316,514,512,0.334630
3,0.3,203,32596,234,567,0.336371
4,0.4,176,32715,115,594,0.331762
5,0.5,155,32757,73,615,0.310621
6,0.6,127,32782,48,643,0.268783
7,0.7,108,32802,28,662,0.238411
8,0.8,89,32810,20,681,0.202503
9,0.9,63,32816,14,707,0.148760


In [11]:
lrn.load_checkpoint()

{'epoch': 10,
 'valid_loss': 0.09144518146977614,
 'valid_score': 0.3094736933708191}

In [12]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,320,31787,1043,450,0.300047
2,0.2,240,32641,189,530,0.400334
3,0.3,195,32742,88,575,0.370370
4,0.4,171,32778,52,599,0.344411
5,0.5,147,32797,33,623,0.309474
6,0.6,133,32805,25,637,0.286638
7,0.7,111,32814,16,659,0.247492
8,0.8,89,32823,7,681,0.205543
9,0.9,64,32825,5,706,0.152563


In [53]:
for param in model.parameters():
    param.requires_grad = True

In [54]:
lrn.name = 'resnest50-mixed-spec-unfrozen-20210502-144336'

In [55]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-52


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-02 17:03:15 epoch:  30 train loss: 0.115589 train f1: 0.549723 valid loss: 0.123151 valid f1: 0.216257


  0%|          | 0/336 [00:00<?, ?it/s]

KeyboardInterrupt: 